#### Import Python packages 

In [100]:
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [101]:
print(os.getcwd())

filepath = os.getcwd() + '/event_data'

for root, dirs, files in os.walk(filepath):
    file_path_list = glob.glob(os.path.join(root,'*'))
    print(file_path_list)

/home/workspace
['/home/workspace/event_data/2018-11-04-events.csv', '/home/workspace/event_data/2018-11-01-events.csv', '/home/workspace/event_data/2018-11-22-events.csv', '/home/workspace/event_data/2018-11-27-events.csv', '/home/workspace/event_data/2018-11-24-events.csv', '/home/workspace/event_data/2018-11-18-events.csv', '/home/workspace/event_data/2018-11-03-events.csv', '/home/workspace/event_data/2018-11-20-events.csv', '/home/workspace/event_data/2018-11-11-events.csv', '/home/workspace/event_data/2018-11-29-events.csv', '/home/workspace/event_data/2018-11-02-events.csv', '/home/workspace/event_data/2018-11-26-events.csv', '/home/workspace/event_data/2018-11-13-events.csv', '/home/workspace/event_data/2018-11-28-events.csv', '/home/workspace/event_data/2018-11-25-events.csv', '/home/workspace/event_data/2018-11-30-events.csv', '/home/workspace/event_data/2018-11-12-events.csv', '/home/workspace/event_data/2018-11-06-events.csv', '/home/workspace/event_data/2018-11-08-events.c

#### Processing files to create the master data file csv that will be used for Apache Casssandra tables

In [102]:
full_data_rows_list = [] 
    
for f in file_path_list:

    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
       
        for line in csvreader: 
            full_data_rows_list.append(line) 
            
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


#### Count check on master file 

In [ ]:
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

#### Creating a Cluster and creating connection

In [104]:
from cassandra.cluster import Cluster
try: 
    cluster = Cluster(['127.0.0.1'])
    session = cluster.connect()
except Exception as e:
    print(e) 

#### Creating Keyspace

In [105]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Setting Keyspace

In [107]:
try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)


#### Columns needed for the query `artist, song, length, sesion, iteminsession`
#### Table partition by `sessionid` and cluster by `iteminsession`                     

In [108]:
try:
    session.execute("""DROP TABLE IF EXISTS udacity.music_app_history""")
    session.execute("""CREATE TABLE music_app_history (artistname text, songtitle text,
                    songlength float, sessionid int, iteminSession int, 
                    PRIMARY KEY(sessionid, iteminSession))""")
except Exception as e:
    print(e)

In [109]:
file = 'event_datafile_new.csv'
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader)  
    for line in csvreader: 
        query = "INSERT INTO music_app_history(artistname, songtitle, songlength, sessionid, iteminSession)"
        query = query + "VALUES (%s, %s, %s, %s,%s)" 
        session.execute(query, (line[0], line[9],float(line[5]),int(line[8]),int(line[3])))

#### SELECT to verify that the data have been inserted into the table

In [110]:
try:
    rows = session.execute("""select * from music_app_history where sessionid = 338 and iteminSession =4 """)
except Exception as e: 
    print(e)

for row in rows:
    print(row)

Row(sessionid=338, iteminsession=4, artistname='Faithless', songlength=495.30731201171875, songtitle='Music Matters (Mark Knight Dub)')


#### Columns needed for the query `artistname , song , firstname , lastname`
#### Table partition by `userid, sessionid` and cluster by `iteminsession`

In [111]:

try:
    session.execute("""DROP TABLE IF EXISTS udacity.music_app_history1""")
    session.execute(
    """CREATE TABLE IF NOT EXISTS udacity.music_app_history1 
    (artistname text, songtitle text, firstname text, lastname text, userid int, sessionid int,
    iteminSession int, PRIMARY KEY((userid, sessionid),iteminSession))""")
except Exception as e:
    print(e)
try:
    with open(file, encoding = 'utf8') as f:
        csvreader = csv.reader(f)
        next(csvreader)  
        for line in csvreader: 
            query = """INSERT INTO udacity.music_app_history1
            (artistname,songtitle,firstname,lastname,userid ,sessionid, iteminSession)"""
            query = query + "VALUES (%s,%s,%s,%s,%s,%s,%s)" 
            session.execute(query, (line[0],line[9],line[1],line[4],int(line[10]),int(line[8]),int(line[3]) ))
except Exception as e:
    print(e)        

#### SELECT to verify that the data have been inserted into the table

In [112]:
try:
    rows = session.execute("""select artistname , songtitle , firstname , lastname from  udacity.music_app_history1 
    where userid = 10 and sessionid = 182 """)
except Exception as e:
    print(e)
for row in rows:
    print(row)

Row(artistname='Down To The Bone', songtitle="Keep On Keepin' On", firstname='Sylvie', lastname='Cruz')
Row(artistname='Three Drives', songtitle='Greece 2000', firstname='Sylvie', lastname='Cruz')
Row(artistname='Sebastien Tellier', songtitle='Kilometer', firstname='Sylvie', lastname='Cruz')
Row(artistname='Lonnie Gordon', songtitle='Catch You Baby (Steve Pitron & Max Sanna Radio Edit)', firstname='Sylvie', lastname='Cruz')


#### Columns needed for the query `firstname, lastname`
#### Table partition by `song` and cluster by `userid`             

In [114]:
try:
    session.execute("""DROP TABLE IF EXISTS udacity.music_app_history2""")
    session.execute("""CREATE TABLE IF NOT EXISTS udacity.music_app_history2( songtitle text, firstname text, lastname text,
    userid int, PRIMARY KEY(songtitle,userid))""")
except Exception as e:
    print(e)
try:
    with open(file, encoding = 'utf8') as f:
        csvreader = csv.reader(f)
        next(csvreader)  
        for line in csvreader: 
            query = """INSERT INTO udacity.music_app_history2(songtitle, firstname, lastname,userid)"""
            query = query + "VALUES (%s,%s,%s,%s)" 
            session.execute(query, (line[9],line[1],line[4],int(line[10])))
except Exception as e:
    print(e)  

#### SELECT to verify that the data have been inserted into the table

In [115]:
try:
    rows = session.execute("""select firstname , lastname from  udacity.music_app_history2 
    where  songtitle =  'All Hands Against His Own' """)
except Exception as e:
    print(e)
for row in rows:
    print(row)

Row(firstname='Jacqueline', lastname='Lynch')
Row(firstname='Tegan', lastname='Levine')
Row(firstname='Sara', lastname='Johnson')


#### Drop table

In [ ]:
try:
    session.execute("""DROP TABLE IF EXISTS udacity.music_app_history""")
except Exception as e:
    print(e)
    
try:
    session.execute("""DROP TABLE IF EXISTS udacity.music_app_history1""")
except Exception as e:
    print(e)

try:
    session.execute("""DROP TABLE IF EXISTS udacity.music_app_history2""")
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [116]:
session.shutdown()
cluster.shutdown()